**source:**
* https://thedatafrog.com/en/articles/docker-influxdb-grafana/

**resources:**
* https://docs.influxdata.com/influxdb/v1.8/introduction/get-started/
* https://www.influxdata.com/blog/getting-started-python-influxdb/
* https://influxdb-python.readthedocs.io/en/latest/

In [1]:
# instaling influxdb library
!pip install influxdb

In [2]:
from influxdb import InfluxDBClient
from influxdb import exceptions

In [3]:
try:
    client = InfluxDBClient('localhost', 8086)
    client.ping()
    print('Connected to InfluxDB')
except Exception as e:
    print('Connection error: {}'.format(str(e)))

Connected to InfluxDB


In [4]:
# Creating a test db

client.create_database('testdb')

In [7]:
# Let's check the list of databases

client.get_list_database()

[{'name': '_internal'}, {'name': 'testdb'}]

To send data to influxdb we have to use a JSON format to indicate 

1.- Measurement (mandatory field)
2.- Tags (optional field)
3.- Time (optional field. If no time value is given Influx db will use the system's timestamp)
4.- Field (mandatory)

In [8]:
# from influx db python documentation
json_body = [
    {
        "measurement": "cpu_load_short",
        "tags": {
            "host": "server01",
            "region": "us-west"
        },
        "time": "2009-11-10T23:00:00Z",
        "fields": {
            "value": 0.64
        }
    }
]


In [10]:
# write_points function also can take de different fields as parameters. See documentation.

client.write_points(json_body, database='testdb')

True

In [17]:
# Let's check the value was stored correctly

result = client.query('select "host", "region", "value" from cpu_load_short;', database='testdb')

In [19]:
# seems like is working just right
print("Result: {0}".format(result))

Result: ResultSet({'('cpu_load_short', None)': [{'time': '2009-11-10T23:00:00Z', 'host': 'server01', 'region': 'us-west', 'value': 0.64}]})
